In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, LSTM
from tensorflow.keras import optimizers
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.regularizers import l2

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
filename="DB_modified.csv"
dataset = pd.read_csv(filename)
dataset =np.array(dataset)[1:-1]
dataset=np.asfarray(dataset,float)


In [ ]:
X=dataset[:,:4]
X[:,0]=(X[:,0]-np.mean(X[:,0]))/np.std(X[:,0])
X[:,1]=(X[:,1]-np.mean(X[:,1]))/np.std(X[:,1])
X[:,2]=(X[:,2]-np.mean(X[:,2]))/np.std(X[:,2])
X[:,3]=(X[:,3]-np.mean(X[:,3]))/np.std(X[:,3])
y=dataset[:,4:7]

In [ ]:
dfgan = pd.read_csv("fromGAN.csv")
dfgan =np.array(dfgan)
dfgan=np.asfarray(dfgan,float)
xGAN=dfgan[:,0:4]
xGAN[:,0]=(xGAN[:,0]-np.mean(xGAN[:,0]))/np.std(xGAN[:,0])
xGAN[:,1]=(xGAN[:,1]-np.mean(xGAN[:,1]))/np.std(xGAN[:,1])
xGAN[:,2]=(xGAN[:,2]-np.mean(xGAN[:,2]))/np.std(xGAN[:,2])
xGAN[:,3]=(xGAN[:,3]-np.mean(xGAN[:,3]))/np.std(xGAN[:,3])
yGAN=dfgan[:,4:7]

In [ ]:
xtotal=np.vstack((X, xGAN))
ytotal=np.vstack((y, yGAN))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(xtotal, ytotal, test_size=0.15, random_state=42, shuffle=True)
x_train, x_cv, y_train, y_cv = train_test_split(x_train, y_train, test_size=0.15, random_state=42, shuffle=True)

In [ ]:
x_traindim =np.expand_dims(x_train, 1)
y_traindim =np.expand_dims(y_train, 1)
x_cvdim =np.expand_dims(x_cv, 1)
y_cvdim =np.expand_dims(y_cv, 1)
x_testdim =np.expand_dims(x_test, 1)
y_testdim =np.expand_dims(y_test, 1)

In [ ]:
model = Sequential()
model.add(LSTM(32, return_sequences=True, input_shape=(x_traindim[1].shape)))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32, return_sequences=False))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='linear'))
model.compile(loss='mean_squared_error', optimizer="adam", metrics=['MSE'])
model.summary()
checkpointer = ModelCheckpoint( monitor='val_loss',filepath=filepath, verbose=1, save_best_only=True)
history3=model.fit(x_traindim,y_train, validation_data=(x_cvdim, y_cv), epochs=1500, batch_size=16, callbacks=[checkpointer])
minloss=round(min(history3.history['val_loss']), 1) 
df=pd.DataFrame(history3.history)
model.save('.\model.h5')
df.to_csv('.\loss.csv')